# Importing Required Libraries

In [29]:
import pickle
import numpy as np
import nltk
import json
import random
from nltk.stem import WordNetLemmatizer
from tkinter import *

# Importing the created model

In [30]:
from tensorflow.keras.models import load_model
model = load_model('chatbot_model.h5')

# Retrieving the intents, words and classes

In [31]:
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

# Text pre-processing (Tokenizing and lemmatizing each word)

In [32]:
def clean_sentence(sentence):
    
    # Tokenizing the pattern - splitting words into an array
    sentence_words = nltk.word_tokenize(sentence)
    
    # Lemmatizing each word - creating short forms (lemma) for words
    lemmatizer = WordNetLemmatizer()
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Returning the bag of words array = '0' or '1' for each word in the bag of words that exists in the sentence

In [33]:
def bow(sentence, words, show_details=True):
    
    # Tokenizing the pattern
    sentence_words = clean_sentence(sentence)
    
    # Bag of words - matrix of N words which is the vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                
                # Assigning 1 if the current word exists in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

# Defining the prediction classes

In [34]:
def predict_class(sentence, model):
    
    # Filtering out predictions below a threshold of 0.25
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    # Sorting the results by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

# Defining the responses

In [35]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# Defining chatbot responses based on users input queries

In [36]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

# Creating GUI for the Mental Health Chatbot using tkinter

In [37]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12), wrap='word')
    
        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

base = Tk()
base.title("Mental Health Awareness Canada")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

''

# Creating the Chat Window

In [38]:
ChatLog = Text(base, bd=0, bg="#FBFCFC", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

# Binding a Scrollbar to the Chat Window

In [39]:
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

# Creating a 'Send' Button for users to send messages

In [40]:
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#F06292", activebackground="#F06292",fg='#ffffff',
                    command= send )

# Creating the input box for users to enter messages

In [41]:
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial", wrap='word')
EntryBox.bind("<Return>", send)

'2556002531584send'

# Starting the Mental Health Chatbot Application

In [42]:
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=425, width=370)
EntryBox.place(x=6, y=440, height=50, width=250)
SendButton.place(x=245, y=440, height=50)

base.mainloop()

# Some Output Examples from the Mental Health Chatbot